<a href="https://colab.research.google.com/github/rifkybujana/KOPSI/blob/Research%2FModel/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## INSTALLATION

In [ ]:
!pip install tokenizers

     |████████████████████████████████| 3.3MB 5.2MB/s 


In [ ]:
!pip install transformers

     |████████████████████████████████| 2.5MB 5.2MB/s 
     |████████████████████████████████| 901kB 29.4MB/s 


## CODE

### Import BERT model

In [ ]:
from transformers import AutoTokenizer, AutoModel

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p2")

### Analyzing FacQA dataset

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("train_preprocess.csv")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2495 entries, 0 to 2494
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   question   2495 non-null   object
 1   passage    2495 non-null   object
 2   seq_label  2495 non-null   object
dtypes: object(3)
memory usage: 58.6+ KB


In [ ]:
df.describe()

,question,passage,seq_label
count,2495,2495,2495
unique,2412,1216,2077
top,"['Siapa', 'nama', 'presiden', 'Indonesia', 'se...","['Pemilu', 'pertama', 'multikandidat', 'ini', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
freq,7,17,6


In [ ]:
df.head()

,question,passage,seq_label
0,"['Kelompok', 'apakah', 'yang', 'menyatakan', '...","['Lewat', 'telepon', 'ke', 'kantor', 'berita',...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
1,"['Di', 'pulau', 'manakah', ',', 'PT', 'Mutiara...","['Sejak', 'tahun', '1980', '-', 'an', 'di', 'K...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,"['Apakah', 'warna', 'pasir', 'pada', 'pesisir'...","['Titik', 'kedua', 'adalah', 'Pantai', 'Baturu...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
3,"['Apa', 'nama', 'perusahaan', 'China', 'yang',...","['Sichuan', ',', 'Kompas', '-', 'Pemerintah', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,"['Siapa', 'yang', 'akan', 'memindahkan', 'Patu...","['Jakarta', ',', 'Kompas', '-', 'Patung', 'Kar...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."


Jika diperhatikan, dataset yang kita miliki semuanya berbentuk teks, sedangkan seharusnya masing masing kata disimpan dalam sebuah array. Oleh karena itu kita perlu mengubahnya menjadi array terlebih dahulu.

In [ ]:
from ast import literal_eval

In [ ]:
df['question'] = df['question'].apply(literal_eval)
df['passage'] = df['passage'].apply(literal_eval)
df['seq_label'] = df['seq_label'].apply(literal_eval)

In [ ]:
df.head()

,question,passage,seq_label
0,"[Kelompok, apakah, yang, menyatakan, bertanggu...","[Lewat, telepon, ke, kantor, berita, lokal, Cu...","[O, O, O, O, O, O, O, O, O, O, B, I, O, O, O, ..."
1,"[Di, pulau, manakah, ,, PT, Mutiara, Indopearl...","[Sejak, tahun, 1980, -, an, di, Kota, Bau-Bau,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,"[Apakah, warna, pasir, pada, pesisir, pantai, ...","[Titik, kedua, adalah, Pantai, Baturumah, sepa...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,"[Apa, nama, perusahaan, China, yang, bekerja, ...","[Sichuan, ,, Kompas, -, Pemerintah, Indonesia,...","[O, O, O, O, O, O, O, O, O, B, I, I, I, O, O, ..."
4,"[Siapa, yang, akan, memindahkan, Patung, Karti...","[Jakarta, ,, Kompas, -, Patung, Kartini, di, J...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [ ]:
' '.join(df['passage'][0])

'Lewat telepon ke kantor berita lokal Current News Service , Hezb-ul Mujahedeen , kelompok militan Kashmir yang terbesar , menyatakan bertanggung jawab atas ledakan di Srinagar .'

Bisa dilihat, seq_label yang kita miliki terbentuk dari O, B, dan I. Kita harus mengubahnya menjadi bentuk teks yang lebih mudah dibaca

In [ ]:
def read_seq(seq_label):
    result = []

    for i, token in enumerate(seq_label):
        if token != 'O':
            result.append(i)

    return result

In [ ]:
df['seq_label'] = df['seq_label'].apply(read_seq)

In [ ]:
df.head()

,question,passage,seq_label
0,"[Kelompok, apakah, yang, menyatakan, bertanggu...","[Lewat, telepon, ke, kantor, berita, lokal, Cu...","[10, 11]"
1,"[Di, pulau, manakah, ,, PT, Mutiara, Indopearl...","[Sejak, tahun, 1980, -, an, di, Kota, Bau-Bau,...","[30, 31]"
2,"[Apakah, warna, pasir, pada, pesisir, pantai, ...","[Titik, kedua, adalah, Pantai, Baturumah, sepa...","[19, 20]"
3,"[Apa, nama, perusahaan, China, yang, bekerja, ...","[Sichuan, ,, Kompas, -, Pemerintah, Indonesia,...","[9, 10, 11, 12]"
4,"[Siapa, yang, akan, memindahkan, Patung, Karti...","[Jakarta, ,, Kompas, -, Patung, Kartini, di, J...","[18, 19, 20]"


In [ ]:
for i in range(len(df['seq_label'])):
    df['seq_label'][i] = df['passage'][i][df['seq_label'][i][0] : df['seq_label'][i][-1] + 1]

In [ ]:
df.head()

,question,passage,seq_label
0,"[Kelompok, apakah, yang, menyatakan, bertanggu...","[Lewat, telepon, ke, kantor, berita, lokal, Cu...","[Hezb-ul, Mujahedeen]"
1,"[Di, pulau, manakah, ,, PT, Mutiara, Indopearl...","[Sejak, tahun, 1980, -, an, di, Kota, Bau-Bau,...","[Pulau, Makassar]"
2,"[Apakah, warna, pasir, pada, pesisir, pantai, ...","[Titik, kedua, adalah, Pantai, Baturumah, sepa...","[putih, halus]"
3,"[Apa, nama, perusahaan, China, yang, bekerja, ...","[Sichuan, ,, Kompas, -, Pemerintah, Indonesia,...","[Sinopec, Southwest, Petroleum, Biro]"
4,"[Siapa, yang, akan, memindahkan, Patung, Karti...","[Jakarta, ,, Kompas, -, Patung, Kartini, di, J...","[Pemerintah, DKI, Jakarta]"


In [ ]:
df_join = df.copy()

In [ ]:
df_join['question'] = df_join['question'].apply(' '.join)

In [ ]:
df_join['passage'] = df_join['passage'].apply(' '.join)

In [ ]:
df_join['seq_label'] = df_join['seq_label'].apply(' '.join)

In [ ]:
df_join.head()

,question,passage,seq_label
0,Kelompok apakah yang menyatakan bertanggung ja...,Lewat telepon ke kantor berita lokal Current N...,Hezb-ul Mujahedeen
1,"Di pulau manakah , PT Mutiara Indopearl berlok...",Sejak tahun 1980 - an di Kota Bau-Bau terdapat...,Pulau Makassar
2,Apakah warna pasir pada pesisir pantai Baturum...,Titik kedua adalah Pantai Baturumah sepanjang ...,putih halus
3,Apa nama perusahaan China yang bekerja sama de...,"Sichuan , Kompas - Pemerintah Indonesia dan Ch...",Sinopec Southwest Petroleum Biro
4,Siapa yang akan memindahkan Patung Kartini dal...,"Jakarta , Kompas - Patung Kartini di Jalan Tam...",Pemerintah DKI Jakarta


### Test Question Answering

In [ ]:
from transformers import pipeline

In [ ]:
question = df_join['question']
context = df_join['passage']

In [ ]:
qa = pipeline("question-answering")

In [ ]:
answer = qa(question=question, context=context)